In [1]:
#%%file kmerseekvisualization.py
# This document is the jupyter file that shows the results of the functions made by this file
# The python file sigseq has a function called get_overlapping_kmers which outputs a list of overlaps
# This is an example of an overlap. It includes the index, hp, sequence, hashval : (30, 'phhphhhhhhhh', 'DAGDVGAAPPGA', 14427879530792476309)
# The list orders each index from smallest to largest
# This file creates a table so that the sequences are paired to eachother
# The visualization takes in the table that displays the overlaps to create the visualization
# This jupyter notebook creates a python file using magics
# The command that creates the file has been turned into a comment so that a python file is not created everytime it is ran


In [2]:
#%%writefile -a kmerseekvisualization.py
import pandas as pd
import seaborn as sns
from sig2kmer import degenerate_protein_chatgpt
import sourmash
import jupyter_black
import math as math
from rich.console import Console

console = Console()

jupyter_black.load()

### This section came from 24-kmerseek-kmer-visualization.ipynb and computes the signatures. 
### It creates the needed indexes and hashvals using sourmash

In [3]:
ced9_seq = "MTRCTADNSLTNPAYRRRTMATGEMKEFLGIKGTEPTDFGINSDAQDLPSPSRQASTRRMSIGESIDGKINDWEEPRLDIEGFVVDYFTHRIRQNGMEWFGAPGLPCGVQPEHEMMRVMGTIFEKKHAENFETFCEQLLAVPRISFSLYQDVVRTVGNAQTDQCPMSYGRLIGLISFGGFVAAKMMESVELQGQVRNLFVYTSLFIKTRIRNNWKEHNRSWDDFMTLGKQMKEDYERAEAEKVGRRKQNRRWSMIGAGVTAGAIGIVGVVVCGRMMFSLK"
bcl2_seq = "MAHAGRTGYDNREIVMKYIHYKLSQRGYEWDAGDVGAAPPGAAPAPGIFSSQPGHTPHPAASRDPVARTSPLQTPAAPGAAAGPALSPVPPVVHLTLRQAGDDFSRRYRRDFAEMSSQLHLTPFTARGRFATVVEELFRDGVNWGRIVAFFEFGGVMCVESVNREMSPLVDNIALWMTEYLNRHLHTWIQDNGGWDAFVELYGPSMRPLFDFSWLSLKTLLSLALVGACITLGAYLGHK"

In [4]:
from sig2kmer import degenerate_protein_chatgpt

ced9_seq_hp = degenerate_protein_chatgpt(ced9_seq, "hp")
ced9_seq_hp

'hpppphppphpphhhpppphhphphpphhhhphpphpphhhppphpphhphppphpppphphhpphphphpphpphphphphhhhphhppphppphhphhhhhhhhphhphppphhphhhphhpppphpphpphppphhhhhphphphhpphhpphhphppppphhphhphhhhhphhhhhhhphhpphphphphpphhhhpphhhppphppphpppppphpphhphhpphppphpphphpphhppppppphphhhhhhphhhhhhhhhhhphphhhphp'

In [5]:
bcl2_seq_hp = degenerate_protein_chatgpt(bcl2_seq, "hp")
bcl2_seq_hp

'hhphhpphhpppphhhphhphphppphhphphhphhhhhhhhhhhhhhhppphhpphphhhppphhhppphhpphhhhhhhhhhhhphhhhhhphphpphhpphppphppphhphppphphphhphphphhphhpphhpphhphhphhhhhphhhhhphpphppphphhhpphhhhhpphhppphpphhppphhhphhhphhhhphphhhphphhphpphhphhhhhhphphhhhhhpp'

In [6]:
%%file ced9.fasta
ced9
MTRCTADNSLTNPAYRRRTMATGEMKEFLGIKGTEPTDFGINSDAQDLPSPSRQASTRRMSIGESIDGKINDWEEPRLDIEGFVVDYFTHRIRQNGMEWFGAPGLPCGVQPEHEMMRVMGTIFEKKHAENFETFCEQLLAVPRISFSLYQDVVRTVGNAQTDQCPMSYGRLIGLISFGGFVAAKMMESVELQGQVRNLFVYTSLFIKTRIRNNWKEHNRSWDDFMTLGKQMKEDYERAEAEKVGRRKQNRRWSMIGAGVTAGAIGIVGVVVCGRMMFSLK

Overwriting ced9.fasta


In [7]:
%%file bcl2.fasta
bcl2
MAHAGRTGYDNREIVMKYIHYKLSQRGYEWDAGDVGAAPPGAAPAPGIFSSQPGHTPHPAASRDPVARTSPLQTPAAPGAAAGPALSPVPPVVHLTLRQAGDDFSRRYRRDFAEMSSQLHLTPFTARGRFATVVEELFRDGVNWGRIVAFFEFGGVMCVESVNREMSPLVDNIALWMTEYLNRHLHTWIQDNGGWDAFVELYGPSMRPLFDFSWLSLKTLLSLALVGACITLGAYLGHK

Overwriting bcl2.fasta


In [8]:
ksizes = 12, 13
ced9_bcl2_fastas = {"ced9": "ced9.fasta", "bcl2": "bcl2.fasta"}
ced9_bcl2_sigfiles = {}


for ksize in ksizes:
    for name, fasta in ced9_bcl2_fastas.items():
        param_string = f"hp,scaled=1,k={ksize}"
        sig = f"{name}.hp.k{ksize}.sig"
        ! sourmash sketch protein -p $param_string --name $name $fasta -o $sig
        ced9_bcl2_sigfiles[(name, ksize)] = sig


== This is sourmash version 4.9.4. ==
== Please cite Irber et. al (2024), doi:10.21105/joss.06830. ==

computing signatures for files: ced9.fasta
Computing a total of 1 signature(s) for each input.
... reading sequences from ced9.fasta
Traceback (most recent call last):
  File "/home/delilah/miniconda3/bin/sourmash", line 8, in <module>
    sys.exit(main())
             ~~~~^^
  File "/home/delilah/miniconda3/lib/python3.13/site-packages/sourmash/__main__.py", line 20, in main
    retval = mainmethod(args)
  File "/home/delilah/miniconda3/lib/python3.13/site-packages/sourmash/cli/sketch/protein.py", line 110, in main
    return sourmash.command_sketch.protein(args)
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/home/delilah/miniconda3/lib/python3.13/site-packages/sourmash/command_sketch.py", line 294, in protein
    _execute_sketch(args, signatures_factory)
    ~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/delilah/miniconda3/lib/python3.13/site-packages/sourmash/c

In [9]:
ksizes = 12, 13
ced9_bcl2_sigs = dict.fromkeys(ksizes, {})

for (name, ksize), sigfile in ced9_bcl2_sigfiles.items():
    ced9_bcl2_sigs[ksize][name] = list(sourmash.load_file_as_signatures(sigfile))[0]
ced9_bcl2_sigs

{12: {'ced9': SourmashSignature('ced9', 8a102fcb),
  'bcl2': SourmashSignature('bcl2', 6d766ffa)},
 13: {'ced9': SourmashSignature('ced9', 8a102fcb),
  'bcl2': SourmashSignature('bcl2', 6d766ffa)}}

In [10]:
from sigseq import SigSeq

# this shows that there was an issue with creating a visualization in SigSeq when there was more than 1 match

for ksize, sigs in ced9_bcl2_sigs.items():
    print(f"ksize: {ksize}")

    ced9_sigseq = SigSeq(sigs["ced9"], ced9_seq)
    bcl2_sigseq = SigSeq(sigs["bcl2"], bcl2_seq)

    try:
        ced9_sigseq.display_alignment(bcl2_sigseq)
    except ValueError as e:
        print(e)
        continue

ksize: 12
Non-sequential indices -- Previous: 32, current: 75
ksize: 13
Non-sequential indices -- Previous: 32, current: 75


# This is where the code for the visualization feature starts

In [11]:
# %%writefile -a kmerseekvisualization.py

# This function displays the aligment table

# It takes in the list created by the function get_overlapping_kmers from the Python file SigSeq,
# the names of the sequences, and the alphabet that was used
# For V2 we may want to rename this to Target and Query


import pandas as pd


def AlignmentTable(
    aligmentdatalist1: list,
    aligmentName1: str,
    aligmentdatalist2: list,
    aligmentName2: str,
    alphabet: str,
):

    columna = [
        aligmentName1 + " Index",
        alphabet,
        aligmentName1 + " protein sequence",
        "hashval",
    ]
    columnb = [
        aligmentName2 + " Index",
        alphabet,
        aligmentName2 + " protein sequence",
        "hashval2",
    ]

    sequencea = pd.DataFrame(aligmentdatalist1, columns=columna)
    sequenceb = pd.DataFrame(aligmentdatalist2, columns=columnb).drop(
        columns=["hashval2"]
    )

    return sequencea.merge(sequenceb, on=alphabet, how="left")

In [12]:
bcl2_overlap = ced9_sigseq.get_overlapping_kmers(bcl2_sigseq)
bcl2_overlap

[(31, 'hhphhhhhhhhhh', 'AGDVGAAPPGAAP', 3645756411471613889),
 (32, 'hphhhhhhhhhhh', 'GDVGAAPPGAAPA', 2514714370726888542),
 (75, 'hhhhhhhhhhhph', 'AAPGAAAGPALSP', 15031373936551915420),
 (80, 'hhhhhhphhhhhh', 'AAGPALSPVPPVV', 13577974435142756968),
 (130, 'hphhpphhpphhp', 'ATVVEELFRDGVN', 5129211217928395354),
 (131, 'phhpphhpphhph', 'TVVEELFRDGVNW', 8826590022931992754),
 (138, 'pphhphhphhhhh', 'RDGVNWGRIVAFF', 6527671935629467448),
 (139, 'phhphhphhhhhp', 'DGVNWGRIVAFFE', 7341826119630988800),
 (140, 'hhphhphhhhhph', 'GVNWGRIVAFFEF', 16460362414207973602),
 (141, 'hphhphhhhhphh', 'VNWGRIVAFFEFG', 9587247570793616415),
 (142, 'phhphhhhhphhh', 'NWGRIVAFFEFGG', 5707605181796643975),
 (143, 'hhphhhhhphhhh', 'WGRIVAFFEFGGV', 657816069765105719),
 (144, 'hphhhhhphhhhh', 'GRIVAFFEFGGVM', 16658868943958856360),
 (200, 'hhhhphphhhphp', 'LYGPSMRPLFDFS', 13918125851660451317)]

In [13]:
ced9_overlap = bcl2_sigseq.get_overlapping_kmers(ced9_sigseq)
ced9_overlap

[(145, 'hphhpphhpphhp', 'FSLYQDVVRTVGN', 5129211217928395354),
 (146, 'phhpphhpphhph', 'SLYQDVVRTVGNA', 8826590022931992754),
 (162, 'pphhphhphhhhh', 'QCPMSYGRLIGLI', 6527671935629467448),
 (163, 'phhphhphhhhhp', 'CPMSYGRLIGLIS', 7341826119630988800),
 (164, 'hhphhphhhhhph', 'PMSYGRLIGLISF', 16460362414207973602),
 (165, 'hphhphhhhhphh', 'MSYGRLIGLISFG', 9587247570793616415),
 (166, 'phhphhhhhphhh', 'SYGRLIGLISFGG', 5707605181796643975),
 (167, 'hhphhhhhphhhh', 'YGRLIGLISFGGF', 657816069765105719),
 (168, 'hphhhhhphhhhh', 'GRLIGLISFGGFV', 16658868943958856360),
 (253, 'hhhhhhphhhhhh', 'MIGAGVTAGAIGI', 13577974435142756968),
 (257, 'hhphhhhhhhhhh', 'GVTAGAIGIVGVV', 3645756411471613889),
 (258, 'hphhhhhhhhhhh', 'VTAGAIGIVGVVV', 2514714370726888542),
 (260, 'hhhhhhhhhhhph', 'AGAIGIVGVVVCG', 15031373936551915420),
 (267, 'hhhhphphhhphp', 'GVVVCGRMMFSLK', 13918125851660451317)]

In [14]:
df = AlignmentTable(bcl2_overlap, "bcl2", ced9_overlap, "ced9", "hp")
df

,bcl2 Index,hp,bcl2 protein sequence,hashval,ced9 Index,ced9 protein sequence
0,31,hhphhhhhhhhhh,AGDVGAAPPGAAP,3645756411471613889,257,GVTAGAIGIVGVV
1,32,hphhhhhhhhhhh,GDVGAAPPGAAPA,2514714370726888542,258,VTAGAIGIVGVVV
2,75,hhhhhhhhhhhph,AAPGAAAGPALSP,15031373936551915420,260,AGAIGIVGVVVCG
3,80,hhhhhhphhhhhh,AAGPALSPVPPVV,13577974435142756968,253,MIGAGVTAGAIGI
4,130,hphhpphhpphhp,ATVVEELFRDGVN,5129211217928395354,145,FSLYQDVVRTVGN
5,131,phhpphhpphhph,TVVEELFRDGVNW,8826590022931992754,146,SLYQDVVRTVGNA
6,138,pphhphhphhhhh,RDGVNWGRIVAFF,6527671935629467448,162,QCPMSYGRLIGLI
7,139,phhphhphhhhhp,DGVNWGRIVAFFE,7341826119630988800,163,CPMSYGRLIGLIS
8,140,hhphhphhhhhph,GVNWGRIVAFFEF,16460362414207973602,164,PMSYGRLIGLISF
9,141,hphhphhhhhphh,VNWGRIVAFFEFG,9587247570793616415,165,MSYGRLIGLISFG


In [15]:
# %%writefile -a kmerseekvisualization.py
# This feature uses console.print from rich.console. Every match displays a different color.
# This function is used to provide a list of colors in a way that is setup for console.print
# These colors came from matplotlib's Tab20
def colors():
    darkblue = "[bold rgb(31,119,180)]"
    lightblue = "[bold rgb(174,199,232)]"
    darkorange = "[bold rgb(255,127,14)]"
    lightorange = "[bold rgb(255,187,120)]"
    darkgreen = "[bold rgb(44,160,44)]"
    lightgreen = "[bold rgb(152,223,138)]"
    darkred = "[bold rgb(214,39,40)]"
    lightred = "[bold rgb(255,152,150)]"
    darkpurple = "[bold rgb(148,103,189)]"
    lightpurple = "[bold rgb(197,176,213)]"
    darkbrown = "[bold rgb(140,86,75)]"
    lightbrown = "[bold rgb(196,156,148)]"
    darkpink = "[bold rgb(227,119,194)]"
    lightpink = "[bold rgb(247,182,210)]"
    darkgray = "[bold rgb(127,127,127)]"
    lightgray = "[bold rgb(199,199,199)]"
    darkolive = "[bold rgb(188,189,34)]"
    lightolive = "[bold rgb(219,219,141)]"
    darkturquoise = "[bold rgb(23,190,207)]"
    lightturquoise = "[bold rgb(158,218,229)]"
    color_list = [
        darkblue,
        darkorange,
        darkgreen,
        darkred,
        darkpurple,
        darkbrown,
        darkpink,
        darkgray,
        darkolive,
        darkturquoise,
        lightblue,
        lightorange,
        lightgreen,
        lightred,
        lightpurple,
        lightbrown,
        lightpink,
        lightgray,
        lightolive,
        lightturquoise,
    ]

    return color_list

In [16]:
dark_purple = colors()[4]
console.print(dark_purple + "Color coded matches")

Color coded matches

In [17]:
# %%writefile -a kmerseekvisualization.py
# This function adds empty characters to the end of the sequence
# This allows for the sequence to be printed out in chunks of 10
# For V2 we may want to reconsider how the last 50 characters of a sequence is displayed
def corrected_length(sequence, iterations):
    length_of_sequence = len(sequence)
    total_page = 50 * iterations
    blank_letters = total_page - length_of_sequence
    sequence = sequence + " " * blank_letters
    return sequence

In [18]:
# %%writefile -a kmerseekvisualization.py
# This function makes the counter display for the feature 10,20,30,40,50
# Iterations means how many lines will the visualization be
# 1 iterations is 50 characters
# In V2 we may want to fix what happens at character 100


def sequence_counter(iteration):
    if iteration == 0 or iteration == 1:
        blank = " " * 8
    elif 1 < iteration < 20:
        blank = " " * 7
    elif iteration >= 20:
        blank = " " * 6
    block = iteration * 50
    return str(
        blank
        + str(10 + block)
        + " "
        + blank
        + str(20 + block)
        + " "
        + blank
        + str(30 + block)
        + " "
        + blank
        + str(40 + block)
        + " "
        + blank
        + str(50 + block)
        + " "
    )

In [19]:
# %%writefile -a kmerseekvisualization.py
# this function is for both the sequence and its HP
# This prints the sequences out in chunks of 10 until it hits 50 characters
def display_sequence(sequence, iteration):
    block = iteration * 50
    return (
        sequence[0 + block : 10 + block]
        + " "
        + sequence[10 + block : 20 + block]
        + " "
        + sequence[20 + block : 30 + block]
        + " "
        + sequence[30 + block : 40 + block]
        + " "
        + sequence[40 + block : 50 + block]
        + " "
    )

In [20]:
# %%writefile -a kmerseekvisualization.py
# The row cuts off at 50 characters
# This functions defines the cutoff points for each line
def cutoffpoints(iterations):
    cutoffs = []
    for i in range(iterations):
        cutoffs.append(51 + (i * 50))
    return cutoffs

In [21]:
# %%writefile -a kmerseekvisualization.py
# This created the (Position: x:y, length) part of the string that is attached to the match of the string
# This function takes in the table created by
# the table have these values matched
def end_of_match(df, row):
    column_names = list(df.columns)
    index = df.iloc[row][column_names[0]]
    sequence = df.iloc[row][column_names[2]]
    end = (
        " (Position "
        + str(index)
        + ":"
        + str(index + len(sequence))
        + ", len:"
        + str(len(sequence))
        + ")"
    )
    return end

In [22]:
end_of_match(df, 4)

' (Position 130:143, len:13)'

In [23]:
# %%writefile -a kmerseekvisualization.py
# This splits the string if it will span over 2 rows.
# This takes in the total number of rows in the table
# This requires the uses of lists and tuples because each match needs to be the same color
# Not all matches have to be split so this keeps everything uniform and able to be tagged with the matching color


def split_match(df, iterations, row):
    column_names = list(df.columns)
    # start_position is the number that is displayed in the column __Index. For example the match could start at position 48
    start_position = df.iloc[row][column_names[4]]
    # Each row only allows for 50 characters.
    # The end position is where it starts(48) + the length of the match.
    # If the match will go beyond the allowed 50 characters it will need to be split
    end_position = start_position + len(df.iloc[row][column_names[2]])
    sequence = df.iloc[row][column_names[2]]
    splits = []
    needs_to_split = []
    for i in cutoffpoints(iterations):
        # If a cutoff point [51,101,151] is in the middle of a match it will need to be broken up
        if start_position < i < end_position:
            # This determines how many characters will be displayed in the first row
            how_many_first_row = i - start_position
            # this adds the information at the end of the match
            end_string = end_of_match(df, row)
            first_row = [start_position, (sequence[0:how_many_first_row], "")]
            second_row = [
                start_position + how_many_first_row,
                (sequence[how_many_first_row:], end_string),
            ]
            splits.append(first_row)
            splits.append(second_row)
            needs_to_split = "True"
            break
    if needs_to_split != "True":
        splits.append([start_position, (sequence, end_of_match(df, row))])
    return splits

In [24]:
for i in range(len(df)):
    print(split_match(df, 6, i))

[[np.int64(257), ('AGDVGAAPPGAAP', ' (Position 31:44, len:13)')]]
[[np.int64(258), ('GDVGAAPPGAAPA', ' (Position 32:45, len:13)')]]
[[np.int64(260), ('AAPGAAAGPALSP', ' (Position 75:88, len:13)')]]
[[np.int64(253), ('AAGPALSPVPPVV', ' (Position 80:93, len:13)')]]
[[np.int64(145), ('ATVVEE', '')], [np.int64(151), ('LFRDGVN', ' (Position 130:143, len:13)')]]
[[np.int64(146), ('TVVEE', '')], [np.int64(151), ('LFRDGVNW', ' (Position 131:144, len:13)')]]
[[np.int64(162), ('RDGVNWGRIVAFF', ' (Position 138:151, len:13)')]]
[[np.int64(163), ('DGVNWGRIVAFFE', ' (Position 139:152, len:13)')]]
[[np.int64(164), ('GVNWGRIVAFFEF', ' (Position 140:153, len:13)')]]
[[np.int64(165), ('VNWGRIVAFFEFG', ' (Position 141:154, len:13)')]]
[[np.int64(166), ('NWGRIVAFFEFGG', ' (Position 142:155, len:13)')]]
[[np.int64(167), ('WGRIVAFFEFGGV', ' (Position 143:156, len:13)')]]
[[np.int64(168), ('GRIVAFFEFGGVM', ' (Position 144:157, len:13)')]]
[[np.int64(267), ('LYGPSMRPLFDFS', ' (Position 200:213, len:13)')]]


In [25]:
# %%writefile -a kmerseekvisualization.py
# The sequence is displayed in chunks of 10. This function makes sure that the match has the same spacing to stay aligned
def spaces(string1, string2):
    position = 0
    string = ""
    # There is 55 characters 50+5 spaces per row
    for i in range(60):
        try:
            # If the sequence has a space, add a space
            if string1[i] == " ":
                string += " "
            else:
                string += string2[position]
                position = position + 1
                # once the match runs out of characters it means that the spaces have been fully added
        except IndexError:
            break

    return string

In [26]:
# %%writefile -a kmerseekvisualization.py
# This function pairs the colors properly, even if they go into differnt lines
# This also flattens out the list of lists a little bit
# In V2 we may want to incorporate this function into split_match and have the colors be added there


def colorpairs(df, iteration):
    matches = []
    for i in range(len(df)):
        matches.append(split_match(df, iteration, i))
    color_matched_matches = []

    possible_colors = colors()
    for i in range(len(matches)):
        if len(matches[i]) == 2:
            matches[i][0].append(possible_colors[i])
            matches[i][1].append(possible_colors[i])
            color_matched_matches.append(matches[i][0])
            color_matched_matches.append(matches[i][1])

        else:
            matches[i][0].append(possible_colors[i])
            color_matched_matches.append(matches[i][0])
    return color_matched_matches

In [27]:
# %%writefile -a kmerseekvisualization.py
# This function adds blank spaces to the matches so that they stay in alignment with the main sequence


def match_placements(df, total_iterations, current_iteration):
    color_pairs = colorpairs(df, total_iterations)
    section = []
    start_position = 0 + (current_iteration * 50)
    end_position = 50 + (current_iteration * 50)
    for i in range(len(color_pairs)):
        # If there is a match within the 50 characters of each row spaces are added to the front of the matches
        if start_position < color_pairs[i][0] < end_position:
            color_matched = [
                " " * (color_pairs[i][0] - start_position - 1) + color_pairs[i][1][0],
                color_pairs[i][1][1],
            ]
            section.append([color_matched, color_pairs[i][2]])
        else:
            continue
        # This there is not a match on this row it skips over it
    if section == []:
        section.append(["  ", "  "])
    return section

In [28]:
# %%writefile -a kmerseekvisualization.py
# This function is to group the matches by their colors and text
def groups(df, total_iterations, current_iteration):
    group = []
    for i in range(len(match_placements(df, total_iterations, current_iteration))):
        group.append(
            match_placements(df, total_iterations, current_iteration)[i][1]
            + spaces(
                display_sequence(ced9_seq, current_iteration),
                match_placements(df, total_iterations, current_iteration)[i][0][0],
            )
            + match_placements(df, total_iterations, current_iteration)[i][0][1]
        )
    return group

# This is the Final result

In [29]:
aligmentdatalist1 = bcl2_overlap
aligmentName1 = "bcl2"
aligmentdatalist2 = ced9_overlap
aligmentName2 = "ced9"
alphabet = "hp"
full_sequence = ced9_seq
full_sequence_converted = ced9_seq_hp

In [30]:
# %%writefile -a kmerseekvisualization.py
console = Console()


# mode means light or dark mode. If it is light mode text is black. If your monitor is in dark mode text is white


def kmerseekvisualization(
    aligmentdatalist1: list,
    aligmentName1: str,
    aligmentdatalist2: list,
    aligmentName2: str,
    alphabet: str,
    full_sequence: str,
    full_sequence_converted: str,
    mode: str,
):
    df = AlignmentTable(
        aligmentdatalist1,
        aligmentName1,
        aligmentdatalist2,
        aligmentName2,
        alphabet,
    )
    total_iterations = math.ceil(len(full_sequence) / 50)
    sequence = corrected_length(full_sequence, total_iterations)
    converted_sequence = corrected_length(full_sequence_converted, total_iterations)

    if mode == "dark":
        color = "[white]"

    elif mode == "light":
        color = "[black]"

    visualization_string = []
    for i in range(total_iterations):
        visualization_string.append(color + aligmentName2)
        visualization_string.append(color + sequence_counter(i))
        visualization_string.append(display_sequence(sequence, i))
        visualization_string.append(display_sequence(converted_sequence, i))
        for j in groups(df, total_iterations, i):
            visualization_string.append(j)
        visualization_string.append(" ")

    # This returns a list
    # If you want to call it and not print after, you can change it to console.print("\n".join(visualization_string))
    return visualization_string

# This is the final result

In [31]:
ced9_bcl2_example = kmerseekvisualization(
    bcl2_overlap, "bcl2", ced9_overlap, "ced9", "hp", ced9_seq, ced9_seq_hp, "dark"
)
console.print("\n".join(ced9_bcl2_example))

ced9
        10         20         30         40         50 
MTRCTADNSL TNPAYRRRTM ATGEMKEFLG IKGTEPTDFG INSDAQDLPS 
hpppphppph pphhhpppph hphphpphhh hphpphpphh hppphpphhp 
    
 
ced9
        60         70         80         90         100 
PSRQASTRRM SIGESIDGKI NDWEEPRLDI EGFVVDYFTH RIRQNGMEWF 
hppphpppph phhpphphph pphpphphph phhhhphhpp phppphhphh 
    
 
ced9
       110        120        130        140        150 
GAPGLPCGVQ PEHEMMRVMG TIFEKKHAEN FETFCEQLLA VPRISFSLYQ 
hhhhhhphhp hppphhphhh phhpppphpp hpphppphhh hhphphphhp 
                                                ATVVEE 
                                                 TVVEE 
 
ced9
       160        170        180        190        200 
DVVRTVGNAQ TDQCPMSYGR LIGLISFGGF VAAKMMESVE LQGQVRNLFV 
phhpphhphp pppphhphhp hhhhhphhhh hhhphhpphp hphphpphhh 
LFRDGVN (Position 130:143, len:13)
LFRDGVNW (Position 131:144, len:13)
            RDGVNWGRI VAFF (Position 138:151, len:13)
             DGVNWGRI VAFFE (Position 139:152, len:13)
              GVNWGRI VAFFEF (Position 140:153, len:13)
               VNWGRI VAFFEFG (Position 141:154, len:13)
                NWGRI VAFFEFGG (Position 142:155, len:13)
                 WGRI VAFFEFGGV (Position 143:156, len:13)
                  GRI VAFFEFGGVM  (Position 144:157, len:13)
 
ced9
       210        220        230        240        250 
YTSLFIKTRI RNNWKEHNRS WDDFMTLGKQ MKEDYERAEA EKVGRRKQNR 
hpphhhppph ppphpppppp hpphhphhpp hppphpphph pphhpppppp 
    
 
ced9
       260        270        280        290        300 
RWSMIGAGVT AGAIGIVGVV VCGRMMFSLK                       
phphhhhhhp hhhhhhhhhh hphphhhphp                       
      AGDV GAAPPGAAP (Position 31:44, len:13)
       GDV GAAPPGAAPA  (Position 32:45, len:13)
         A APGAAAGPAL SP (Position 75:88, len:13)
  AAGPALSP VPPVV (Position 80:93, len:13)
                 LYGP SMRPLFDFS (Position 200:213, len:13)
 